In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib_inline.backend_inline
from astropy.table import Table
from astropy.io import fits
from pathlib import Path
import numpy as np
import eazy
import pickle
import gc
import os
import psutil
import time

p = psutil.Process(os.getpid())
# p.nice(-1)

matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

WD = Path('/data1/hbahk/spherex-photoz/spherex-challenge/7ds_challenge')
os.chdir(WD)
TEMPDIR = WD.parent / 'BROWN_COSMOS'

RECALCULATE_TEMPFILT = False

In [3]:
params = {}
params['FILTERS_RES']       = "FILTER.RES+7DTreduced.res"
params['TEMPLATES_FILE']    = "BROWN_COSMOS.template"
params['TEMP_ERR_FILE']     = "../template_error_cosmos2020.txt"
params['TEMP_ERR_A2']       = 0.0 # nullify the template error

params['SYS_ERR']           = 0.01

params['MW_EBV']            = 0.016577
params['CAT_HAS_EXTCORR']   = False

params['CATALOG_FILE']      = "input/input_RIS.csv" 

dir_output = WD/"output"/"output_RIS"

if not dir_output.exists():
    dir_output.mkdir(parents=True)
params['OUTPUT_DIRECTORY']  = dir_output.as_posix()
params['MAIN_OUTPUT_FILE']  = "output"

params['APPLY_PRIOR']       = True
params['PRIOR_FILE']        = "prior_R_zmax7.dat"
params['PRIOR_FILTER']      = 429  # 7DT band corresponding to 6500 Angstrom
params['PRIOR_ABZP']        = 23.9
params['PRIOR_FLOOR']       = 0.01

params['FIX_ZSPEC']         = False
params['Z_MIN']             = 0.015
params['Z_MAX']             = 5.8
params['Z_STEP']            = 0.01
params['Z_STEP_TYPE']       = 1

translate_filename = 'dummy.translate'

params_RIS = params.copy()

In [4]:
params['CATALOG_FILE']      = "input/input_WFS.csv"
dir_output = WD/"output"/"output_WFS"
dir_output.mkdir(parents=True, exist_ok=True)
params['OUTPUT_DIRECTORY']  = dir_output.as_posix()
params_WFS = params.copy()

params['CATALOG_FILE']      = "input/input_IMS.csv"
dir_output = WD/"output"/"output_IMS"
dir_output.mkdir(parents=True, exist_ok=True)
params['OUTPUT_DIRECTORY']  = dir_output.as_posix()
params_IMS = params.copy()

In [9]:
def get_pit(zspec, zgrid, lnp, trdz):
    """
    PIT function for evaluating the calibration of p(z), 
    as described in Tanaka (2017).
    """
    zspec_grid = np.dot(zspec[:,None], np.ones_like(zgrid)[None,:])
    zlim = zspec_grid >= zgrid
    pit = np.dot(np.exp(lnp)*zlim, trdz)
    
    return pit


def get_crps(zspec, zgrid, lnp, trdz):
    """
    CRPS function for evaluating the calibration of p(z), 
    as described in Tanaka (2017).
    """
    from tqdm import trange
    crps= np.empty(len(lnp), dtype=float)
    for i in trange(len(lnp), desc='Calculating CRPS'):
        zzgrid = np.dot(zgrid[:,None], np.ones_like(zgrid)[None,:])
        lnpgrid = np.dot(np.ones_like(zgrid)[:,None], lnp[i][None,:])
        zlim = zzgrid >= zgrid
        h = np.zeros_like(zgrid)
        h[zgrid >= zspec[i]] = 1
        pit = np.dot(np.exp(lnpgrid)*zlim, trdz)
        crps[i] = np.dot((pit-h)**2, trdz)
    
    return crps


def fit_and_save_result_prior(params, tempfilt, rerun=False):
    start = time.time()
    dir_output = Path(params['OUTPUT_DIRECTORY'])

    for i in range(17):
        outtabpath = dir_output/f'output{i:02d}.fits'
        lnptabpath = dir_output/f'lnp{i:02d}.fits'
        
        if (outtabpath.exists() and lnptabpath.exists()) and (not rerun):
            print(f'{i}th batch already exists ==============')
        else:
            print(f'Fitting {i}th batch =====================')
            start_id = i * 10000
            end_id = (i+1) * 10000
            
            ez = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_filename,
                                zeropoint_file=None, params=params, tempfilt=tempfilt, )
            
            if end_id > len(ez.idx):
                end_id = len(ez.idx)

            ez.ZML_WITH_PRIOR = True
            ez.fit_catalog(ez.idx[start_id:end_id], n_proc=16, prior=True)
            ez.ZML_WITH_PRIOR = True
            ez.fit_at_zbest(prior=True, nproc=1)
            
            try:
                zlimits = ez.pz_percentiles(percentiles=[2.5,16,50,84,97.5],
                                                oversample=5)
            except:
                print('Couldn\'t compute pz_percentiles')
                zlimits = np.zeros((ez.NOBJ, 5), dtype=ez.ARRAY_DTYPE) - 1
                
            tab = Table()
            tab['id'] = ez.OBJID
            tab['z_phot'] = ez.zbest
            tab['z_phot_chi2'] = ez.chi2_best
            tab['z025'] = zlimits[:,0]
            tab['z160'] = zlimits[:,1]
            tab['z500'] = zlimits[:,2]
            tab['z840'] = zlimits[:,3]
            tab['z975'] = zlimits[:,4]
            
            pit = ez.PIT(ez.ZSPEC)
            tab['pit'] = pit
            
            crps = get_crps(ez.ZSPEC[start_id:end_id], ez.zgrid, ez.lnp[start_id:end_id], ez.trdz)
            tab['crps'] = np.empty(len(tab), dtype=float)
            tab['crps'][start_id:end_id] = crps
            
            tab[start_id:end_id].write(outtabpath, overwrite=True)
            phdu = fits.PrimaryHDU(data=ez.lnp[start_id:end_id])
            gridhdu = fits.ImageHDU(data=ez.zgrid)
            hdul = fits.HDUList([phdu, gridhdu])
            hdul.writeto(lnptabpath, overwrite=True)
            del ez, tab, phdu, gridhdu, hdul
            gc.collect()
    
    incatpath = Path(params['CATALOG_FILE'])
    base = Table.read(incatpath)
    
    colnames = ['z_phot', 'z_phot_chi2', 'z160', 'z840', 'id']
    for label in colnames:
        base[label] = np.empty(len(base), dtype=float)
        
    for i in range(17):
        start_id = i*10000
        end_id = (i+1)*10000 if i < 16 else len(base)
        
        outtab = Table.read(dir_output/f'output{i:02d}.fits')
        for label in colnames:
            base[label][start_id:end_id] = outtab[label]
    
    base.write(dir_output/'result.fits', overwrite=True)
    
    end = time.time()
    time_taken_hms = time.strftime('%H:%M:%S', time.gmtime(end-start))
    print(f'Finished in {time_taken_hms}')

In [6]:
with open('tempfilt_nored_001.pickle', 'rb') as rfile:
    tempfilt = pickle.load(rfile)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [10]:
fit_and_save_result_prior(params_IMS, tempfilt)

0th batch already exists ==============
1th batch already exists ==============
2th batch already exists ==============
3th batch already exists ==============
4th batch already exists ==============
5th batch already exists ==============
6th batch already exists ==============
7th batch already exists ==============
8th batch already exists ==============
9th batch already exists ==============
10th batch already exists ==============
11th batch already exists ==============
12th batch already exists ==============
13th batch already exists ==============
14th batch already exists ==============
15th batch already exists ==============
16th batch already exists ==============
Finished in 00:00:03


In [11]:
fit_and_save_result_prior(params_WFS, tempfilt)

Fitting 0th batch =====================
Read default param file: /data1/hbahk/envs/main/lib/python3.9/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: input/input_WFS.csv
   >>> NOBJ = 166041
F419 E419 (419): 7DT_4000_fitted
F420 E420 (420): 7DT_4250_fitted
F421 E421 (421): 7DT_4500_fitted
F422 E422 (422): 7DT_4750_fitted
F423 E423 (423): 7DT_5000_fitted
F424 E424 (424): 7DT_5250_fitted
F425 E425 (425): 7DT_5500_fitted
F426 E426 (426): 7DT_5750_fitted
F427 E427 (427): 7DT_6000_fitted
F428 E428 (428): 7DT_6250_fitted
F429 E429 (429): 7DT_6500_fitted
F430 E430 (430): 7DT_6750_fitted
F431 E431 (431): 7DT_7000_fitted
F432 E432 (432): 7DT_7250_fitted
F433 E433 (433): 7DT_7500_fitted
F434 E434 (434): 7DT_7750_fitted
F435 E435 (435): 7DT_8000_fitted
F436 E436 (436): 7DT_8250_fitted
F437 E437 (437): 7DT_8500_fitted
F438 E438 (438): 7DT_8750_fitted
Set sys_err = 0.01 (positive=True)
Read PRIOR_FILE:  prior_R_zmax7.dat


191it [00:07, 25.32it/s]
100%|██████████| 191/191 [00:28<00:00,  6.82it/s]


Compute best fits
fit_best: 3.6 s (n_proc=1,  NOBJ=9890)
Fit 33.2 s (n_proc=16, NOBJ=10000)
fit_best: 3.5 s (n_proc=1,  NOBJ=9890)


Calculating CRPS: 100%|██████████| 10000/10000 [00:07<00:00, 1398.36it/s]


Fitting 1th batch =====================
Read default param file: /data1/hbahk/envs/main/lib/python3.9/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: input/input_WFS.csv
   >>> NOBJ = 166041
F419 E419 (419): 7DT_4000_fitted
F420 E420 (420): 7DT_4250_fitted
F421 E421 (421): 7DT_4500_fitted
F422 E422 (422): 7DT_4750_fitted
F423 E423 (423): 7DT_5000_fitted
F424 E424 (424): 7DT_5250_fitted
F425 E425 (425): 7DT_5500_fitted
F426 E426 (426): 7DT_5750_fitted
F427 E427 (427): 7DT_6000_fitted
F428 E428 (428): 7DT_6250_fitted
F429 E429 (429): 7DT_6500_fitted
F430 E430 (430): 7DT_6750_fitted
F431 E431 (431): 7DT_7000_fitted
F432 E432 (432): 7DT_7250_fitted
F433 E433 (433): 7DT_7500_fitted
F434 E434 (434): 7DT_7750_fitted
F435 E435 (435): 7DT_8000_fitted
F436 E436 (436): 7DT_8250_fitted
F437 E437 (437): 7DT_8500_fitted
F438 E438 (438): 7DT_8750_fitted
Set sys_err = 0.01 (positive=True)
Read PRIOR_FILE:  prior_R_zmax7.dat


...

Fitting 16th batch =====================
Read default param file: /data1/hbahk/envs/main/lib/python3.9/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: input/input_WFS.csv
   >>> NOBJ = 166041
F419 E419 (419): 7DT_4000_fitted
F420 E420 (420): 7DT_4250_fitted
F421 E421 (421): 7DT_4500_fitted
F422 E422 (422): 7DT_4750_fitted
F423 E423 (423): 7DT_5000_fitted
F424 E424 (424): 7DT_5250_fitted
F425 E425 (425): 7DT_5500_fitted
F426 E426 (426): 7DT_5750_fitted
F427 E427 (427): 7DT_6000_fitted
F428 E428 (428): 7DT_6250_fitted
F429 E429 (429): 7DT_6500_fitted
F430 E430 (430): 7DT_6750_fitted
F431 E431 (431): 7DT_7000_fitted
F432 E432 (432): 7DT_7250_fitted
F433 E433 (433): 7DT_7500_fitted
F434 E434 (434): 7DT_7750_fitted
F435 E435 (435): 7DT_8000_fitted
F436 E436 (436): 7DT_8250_fitted
F437 E437 (437): 7DT_8500_fitted
F438 E438 (438): 7DT_8750_fitted
Set sys_err = 0.01 (positive=True)
Read PRIOR_FILE:  prior_R_zmax7.dat


191it [00:07, 25.19it/s]
100%|██████████| 191/191 [00:16<00:00, 11.42it/s]


Compute best fits
fit_best: 2.3 s (n_proc=1,  NOBJ=5985)
Fit 20.8 s (n_proc=16, NOBJ=6041)
fit_best: 2.4 s (n_proc=1,  NOBJ=5985)


Calculating CRPS: 100%|██████████| 6041/6041 [00:06<00:00, 949.46it/s] 


Finished in 00:18:21


In [ ]:
fit_and_save_result_prior(params_RIS, tempfilt)

Fitting 0th batch =====================
Read default param file: /data1/hbahk/envs/main/lib/python3.9/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: input/input_RIS.csv
   >>> NOBJ = 166041
F419 E419 (419): 7DT_4000_fitted
F420 E420 (420): 7DT_4250_fitted
F421 E421 (421): 7DT_4500_fitted
F422 E422 (422): 7DT_4750_fitted
F423 E423 (423): 7DT_5000_fitted
F424 E424 (424): 7DT_5250_fitted
F425 E425 (425): 7DT_5500_fitted
F426 E426 (426): 7DT_5750_fitted
F427 E427 (427): 7DT_6000_fitted
F428 E428 (428): 7DT_6250_fitted
F429 E429 (429): 7DT_6500_fitted
F430 E430 (430): 7DT_6750_fitted
F431 E431 (431): 7DT_7000_fitted
F432 E432 (432): 7DT_7250_fitted
F433 E433 (433): 7DT_7500_fitted
F434 E434 (434): 7DT_7750_fitted
F435 E435 (435): 7DT_8000_fitted
F436 E436 (436): 7DT_8250_fitted
F437 E437 (437): 7DT_8500_fitted
F438 E438 (438): 7DT_8750_fitted
Set sys_err = 0.01 (positive=True)
Read PRIOR_FILE:  prior_R_zmax7.dat


191it [00:07, 24.54it/s]
100%|██████████| 191/191 [00:23<00:00,  8.24it/s]


Compute best fits
fit_best: 3.0 s (n_proc=1,  NOBJ=9360)
Fit 27.9 s (n_proc=16, NOBJ=10000)
fit_best: 3.0 s (n_proc=1,  NOBJ=9360)


Calculating CRPS: 100%|██████████| 10000/10000 [00:13<00:00, 730.72it/s]


Fitting 1th batch =====================
Read default param file: /data1/hbahk/envs/main/lib/python3.9/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: input/input_RIS.csv
   >>> NOBJ = 166041
F419 E419 (419): 7DT_4000_fitted
F420 E420 (420): 7DT_4250_fitted
F421 E421 (421): 7DT_4500_fitted
F422 E422 (422): 7DT_4750_fitted
F423 E423 (423): 7DT_5000_fitted
F424 E424 (424): 7DT_5250_fitted
F425 E425 (425): 7DT_5500_fitted
F426 E426 (426): 7DT_5750_fitted
F427 E427 (427): 7DT_6000_fitted
F428 E428 (428): 7DT_6250_fitted
F429 E429 (429): 7DT_6500_fitted
F430 E430 (430): 7DT_6750_fitted
F431 E431 (431): 7DT_7000_fitted
F432 E432 (432): 7DT_7250_fitted
F433 E433 (433): 7DT_7500_fitted
F434 E434 (434): 7DT_7750_fitted
F435 E435 (435): 7DT_8000_fitted
F436 E436 (436): 7DT_8250_fitted
F437 E437 (437): 7DT_8500_fitted
F438 E438 (438): 7DT_8750_fitted
Set sys_err = 0.01 (positive=True)
Read PRIOR_FILE:  prior_R_zmax7.dat


...

191it [00:07, 24.29it/s]
100%|██████████| 191/191 [00:22<00:00,  8.47it/s]


Compute best fits
fit_best: 3.0 s (n_proc=1,  NOBJ=9384)
Fit 27.3 s (n_proc=16, NOBJ=10000)
fit_best: 2.8 s (n_proc=1,  NOBJ=9384)


Calculating CRPS: 100%|██████████| 10000/10000 [00:09<00:00, 1094.34it/s]


Fitting 3th batch =====================
Read default param file: /data1/hbahk/envs/main/lib/python3.9/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: input/input_RIS.csv
   >>> NOBJ = 166041
F419 E419 (419): 7DT_4000_fitted
F420 E420 (420): 7DT_4250_fitted
F421 E421 (421): 7DT_4500_fitted
F422 E422 (422): 7DT_4750_fitted
F423 E423 (423): 7DT_5000_fitted
F424 E424 (424): 7DT_5250_fitted
F425 E425 (425): 7DT_5500_fitted
F426 E426 (426): 7DT_5750_fitted
F427 E427 (427): 7DT_6000_fitted
F428 E428 (428): 7DT_6250_fitted
F429 E429 (429): 7DT_6500_fitted
F430 E430 (430): 7DT_6750_fitted
F431 E431 (431): 7DT_7000_fitted
F432 E432 (432): 7DT_7250_fitted
F433 E433 (433): 7DT_7500_fitted
F434 E434 (434): 7DT_7750_fitted
F435 E435 (435): 7DT_8000_fitted
F436 E436 (436): 7DT_8250_fitted
F437 E437 (437): 7DT_8500_fitted
F438 E438 (438): 7DT_8750_fitted
Set sys_err = 0.01 (positive=True)
Read PRIOR_FILE:  prior_R_zmax7.dat


191it [00:07, 24.36it/s]
  0%|          | 0/191 [00:00<?, ?it/s]